In [1]:
from IPython.core.display import HTML; display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Image as ipy_im;from IPython.display import display
import config
import matplotlib.pyplot as plt
from imageio import imwrite
import yaml
from fastai.core import Path
from fastai.vision import load_learner, defaults
import torch as torch
%matplotlib inline
%reload_ext autoreload
%autoreload 2
defaults.device = torch.device('cpu')

# helper functions to manage data files and use the widget below
from tagging_tool_utils import *
from image_graph import *
import ipywidgets as widgets
letter_mappings = yaml.load(open('../yaml/letter_maps.yaml'))

<p style="font-size:13px">This notebook will demonstrate the process I have used to generate my data labels. For more info on the whole data generation process please see the read me.</p>

In [2]:
pic_name = 'GK_RDR_PG3_2' # name of the picture to be tagged

In [3]:
gr_path = Path('../greek_pages/page_graphs/')
im_path = Path('../greek_pages/page_images/')
gname = pic_name + '.txt'
im_name = pic_name + '.jpeg'

model_name = 'rn_34.pkl'
config.model = load_learner('../models/', model_name) # classifier for greek and english characters, this model is on my local machine
letter_path = Path('../all_data/lgi_data') # location for data, on local machine
config.letter_dest = letter_path/'pg3_data'
get_letter_dict()

config.img_arr = get_image_array(im_path/im_name)

<ul style="font-size: 14px">
    <li>Get the list of components that weren't tagged automatically (see overview of data generation process). These components represent individual characters, letter parts, or clusters of letters that were connected together in the image.</li>
    <li>Set the config variables used by the helper functions above. These act as global variables throughout the different packages</li>
</ul>

In [6]:
yaml_pth = config.letter_dest/'yaml'
with open(yaml_pth/'new_components.yaml', 'rb') as f:
    new_components = yaml.load(f)
config.components = new_components
config.itr = 0
config.temp_path = yaml_pth
config.rows, config.cols = config.img_arr.shape
config.letter_mappings = letter_mappings

<p style="font-size:13px">Get the boundaries for the letter component in the image. This will be displayed below and tagged by the user</p>

In [7]:
lb, ub, lbr, ubr = get_bounds(config.itr)

<ul style="font-size: 14px">
    <li>Write the component to a temporary image file (This will later be moved to a permanent file when it is tagged)</li>
    <li>Open the image and store it in the image widget's value.</li>
</ul>

In [8]:
imwrite(config.temp_path/'temp_letter.jpg', config.img_arr[lbr:ubr,lb:ub])
file = open(config.temp_path/"temp_letter.jpg", "rb")
image = file.read()
config.image = widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

<ul style="font-size: 14px">
    <li>letter_button: Update Letter, moves the temp image to a permanent image with the correct label (labeled by directory)</li>
    <li>skip_button: Skips to the next letter, only here for convenience, generally no components are skipped even if they are just noise</li>
    <li>context_button: Get Context, shows the surrounding characters to help identify the character if not immediately clear</li>
    <li>reset_context_button: Reset Context, returns the image to it's original form</li>
    <li>output: output used to display for the widget</li>
    <li>letter_options: A list of suggested options for the tag, once the first model has been trained this is filled with the top 3 predicted letters</li>
    <li>textbox: A textbox to enter the image label</li>
    <li>Each button gets an on_click function, defined in tagging_tool_utils</li>
</ul>

In [ ]:
letter_button = widgets.Button(description='Update Letter')
skip_button = widgets.Button(description='Skip')
context_button = widgets.Button(description='Get Context')
reset_context_button = widgets.Button(description='Reset Context')
config.output = widgets.Output()
config.letter_options = widgets.Dropdown(options=[''])
config.letter_options.options =  [''] + get_top_preds(config.temp_path/"temp_letter.jpg")
config.textbox = widgets.Text()
    
letter_button.on_click(update_letter)
skip_button.on_click(skip)
context_button.on_click(get_context)
reset_context_button.on_click(reset_context)
# display
widgets.VBox([config.image,config.textbox,config.letter_options,letter_button,skip_button,context_button,reset_context_button,config.output])

<p style="font-size: 14px">Jupyter won't load the widget in github, so here is an image of what it looks like in practice</p>
<img src="../imgs/widget_img.png" style="width: 700px; height=50px;" />

<p style="font-size:14px">After all the images have been tagged, the new data and labels are reviewed and then sent to the GPU instance to update the letter classifier. See the overview in the readme and the model_training notebook for details</p>

In [ ]:
yaml.dump(letter_mappings,open('nbs/letter_maps.yaml','w')) # maps for greek letters and various characters to suitable directory names